# electricity_market_player

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_player

In [ ]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo.policies import MlpPolicy

from electricity_market.electricity_market_env import ElectricityMarketEnv


In [ ]:
#| export
env = ElectricityMarketEnv({
    "battery_capacity": 10000000,
    "init_state_of_charge": 4000000,
    "init_current_demand_of_electricity": 1000,
    "init_current_price": 100
})

env.reset()

model = PPO(MlpPolicy, env, verbose=0)


model.learn(total_timesteps=10_000)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


mean_reward:-85.09 +/- 735.75


In [7]:
!python -m rl_zoo3.train --algo ppo --env ElectricityMarketEnv-v0 -n 50000 --optimize --n-trials 1000 --n-jobs 2 --gym-packages electricity_market

========== ElectricityMarketEnv-v0 ==========
Seed: 1872311507
Loading hyperparameters from: /Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/rl_zoo3/hyperparams/ppo.yml
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/rl_zoo3/train.py", line 279, in <module>
    train()
  File "/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/rl_zoo3/train.py", line 261, in train
    results = exp_manager.setup_experiment()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/rl_zoo3/exp_manager.py", line 189, in setup_experiment
    unprocessed_hyperparams, saved_hyperparams = self.read_hyperparameters()
                                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [11]:
import optuna
from stable_baselines3 import PPO
from electricity_market.electricity_market_env import ElectricityMarketEnv

def optimize_agent(trial):
    # Define hyperparameters to optimize
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    n_steps = trial.suggest_int('n_steps', 16, 2048, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 256, log=True)

    # Create environment
    env = DummyVecEnv([lambda: ElectricityMarketEnv()])

    # Initialize PPO with trial hyperparameters
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size, verbose=0)

    # Train the model
    model.learn(total_timesteps=int(1e3))

    # Evaluate the model
    total_reward = 0
    obs = env.reset()
    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break

    return total_reward

# Set up Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(optimize_agent, n_trials=10)

# Print best trial
print("Best trial:", study.best_trial)

[I 2025-02-05 03:30:37,013] A new study created in memory with name: no-name-9f75bf3a-7604-4757-b435-8887520f901d
/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 31, but because the `RolloutBuffer` is of size `n_steps * n_envs = 35`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 4
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=35 and n_envs=1)
  warnings.warn(
[I 2025-02-05 03:30:37,797] Trial 0 finished with value: -429.2867126464844 and parameters: {'learning_rate': 0.010783162190824478, 'n_steps': 35, 'batch_size': 31}. Best is trial 0 with value: -429.2867126464844.
/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 61, but because the `RolloutBuffer` is of size `n_steps 

Best trial: FrozenTrial(number=3, state=1, values=[1155.1278076171875], datetime_start=datetime.datetime(2025, 2, 5, 3, 30, 38, 517436), datetime_complete=datetime.datetime(2025, 2, 5, 3, 30, 38, 854018), params={'learning_rate': 0.005550291149425107, 'n_steps': 641, 'batch_size': 135}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'n_steps': IntDistribution(high=2048, log=True, low=16, step=1), 'batch_size': IntDistribution(high=256, log=True, low=16, step=1)}, trial_id=3, value=None)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()